# 산출

In [1]:
import pandas as pd
import pymysql
import datetime
import numpy as np
import scipy.stats as st
import time
import gc
import math
import re#from CLS_MASTER import *
from itertools import combinations
import os
import warnings
warnings.filterwarnings(action = 'ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import connectorx as cx
from glob import glob
import psutil
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy.types import VARCHAR, Integer
import datetime

import connectorx as cx
import pandas as pd

# 업로드

In [2]:
target_date_list = pd.date_range("2025-12-30","2025-12-30")

In [3]:
for idx, target_date in enumerate(target_date_list):
    #옮길 계정 정보
    user = "direct"
    password = "Direct999"
    host = "server5.marketingtool.co.kr"
    port = 60001
    db = "direct"
    
    table_list = ["lgcns_miecsm_daily_output"]
    uri = f'''mysql://{user}:{password}@{host}:{port}/{db}'''

    bucket = pd.DataFrame()
    for table in table_list:
        query = f"""
        SELECT * FROM {db}.{table}
        where 1=1
        and rdate in ("{target_date}")
        """
        tmp = cx.read_sql(conn = uri, query = query)
        bucket = pd.concat([tmp, bucket], axis = 0)
        
    #옮겨질 계정 정보
    sql_user = "direct"
    sql_password = "Direct999"
    sql_host = "server5.marketingtool.co.kr"
    sql_schema = "direct"
    sql_port = 60001
    sql_table = "lgcns_miecsm_daily_output_projection"

    conn = pymysql.connect(host = sql_host,
                           user = sql_user,
                           passwd = sql_password,
                           db = sql_schema,
                           port = sql_port)

    s = f'''mysql+pymysql://{sql_user}:{sql_password}@{sql_host}:{sql_port}/{sql_schema}'''
    # , encoding='utf-8'
    ENGINE = create_engine(s, encoding = "utf-8")
    
    # 동일기간 데이터는 제거
    cursor = conn.cursor()
    min_rtime = bucket["rdate"].min()
    max_rtime = bucket["rdate"].max()
    # 처음 업로드시 발생할 예외문 포함
    query = f'''
    delete from {sql_schema}.{sql_table} where rdate >= "{min_rtime}" and rdate <= "{max_rtime}"  '''
    cursor.execute(query)
    conn.commit()
    
    bucket.to_sql(name = sql_table, con=ENGINE, if_exists='append', index = False,
                        chunksize = 10000) # 테이블에 INSERT
    
    print(target_date)

2025-12-30 00:00:00


# 데이터 삭제

In [ ]:
# 운영계
target_date_list = pd.date_range("2025-11-01", "2025-12-14")

for idx, target_date in enumerate(target_date_list):
    sql_user = "direct"
    sql_password = "Direct999"
    sql_host = "server5.marketingtool.co.kr"
    sql_schema = "direct"
    sql_port = 60001
    sql_table = "lgcns_miecsm_price_table"
    
    conn = pymysql.connect(host = sql_host,
                               user = sql_user,
                               passwd = sql_password,
                               db = sql_schema,
                               port = sql_port)

    s = f'''mysql+pymysql://{sql_user}:{sql_password}@{sql_host}:{sql_port}/{sql_schema}'''
    # , encoding='utf-8'
    ENGINE = create_engine(s, encoding = "utf-8")

    # 동일기간 데이터는 제거
    cursor = conn.cursor()
    # 처음 업로드시 발생할 예외문 포함
    query = f'''
        delete from {sql_schema}.{sql_table} where rdate = "{target_date}" '''
    cursor.execute(query)
    conn.commit()
    print(target_date)